## Задание №1. Рекомендательная система фильмов на данных MovieLens 
## Выполнила: Соболева Дарья
## Студентка 317 группы ВМК МГУ
## Версия Python 3.5

## Введение

Одним из бурно развивающихся направлений совершенствования прикладных информационных технологий является развертывание рекомендательных систем -- инструментов автоматической генерации предложений по услугам на основе изучения персональных потребностей клиентов.

<b>Объект исследования:</b> рекомендательная система, построенная на данных MovieLens.
    
<b>Предмет исследования:</b> алгоритмы выбора релевантных рекомендаций.
    
<b>Задачи</b>: предсказание оценки пользователя неизвестному фильму.
    Нахождение небольшого числа фильмов, которые скорее всего заинтересуют конкретного пользователя, используя информацию о предыдущей его аквтиности и характеристиках фильмов.
    
В задачи также входит выявление других данных, которые помогли бы улучшить механизм рекомендаций.

<b>Метрика качества</b>: Mean Squared Error (MSE)

<b>Цель:</b> исследование алгоритмов для рекомендательных систем, позволяющих выбирать рекомендации в условиях большого числа пользователей при неполной или отсутствующей информации об их предпочтениях.

В данной работе будут рассмотрены два возможных подхода:
* <b>Content-based</b> 
* <b>Collaborative filtering</b>

В задаче коллаборативной фильтрации будут рассмотрены <b>neighborhood</b> и <b>latent factor</b> подходы.

In [5]:
import zipfile
import pandas as pd
import numpy as np
from IPython.display import display
import time
from collections import OrderedDict

from content_based import build_categorical, RidgeRegression
from latent_factor import matrix_factorization
from cross_validation import train_test_split, cross_val_score, mean_squared_error

In [2]:
with zipfile.ZipFile('ml-1m.zip', 'r') as myzip:

    with myzip.open('ml-1m/movies.dat', 'r') as r:
        movies = pd.read_csv(r, header=None, encoding='latin_1', sep=b'\s*\::', engine='python')
        movies.columns = ['MovieID', 'Title', 'Genres']
        movies.Title = movies.Title.str.decode("latin_1")
        movies.Genres = movies.Genres.str.decode("latin_1")
        
    with myzip.open('ml-1m/ratings.dat', 'r') as r:
        ratings = pd.read_csv(r, header=None, encoding='latin_1', sep=b'\s*\::', engine='python')
        ratings.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
        ratings.Rating /= 5.0
        
    with myzip.open('ml-1m/users.dat', 'r') as r:
        users = pd.read_csv(r, header=None, encoding='latin_1', sep=b'\s*\::', engine='python')
        users.columns = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']

In [7]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,1.0,978300760
1,1,661,0.6,978302109
2,1,914,0.6,978301968
3,1,3408,0.8,978300275
4,1,2355,1.0,978824291


Для проведения экспериментов предлагается выделить <b>80%</b> оценок пользователей под обучающую выборку, а оставшиеся <b>20%</b> 
использовать в качестве контрольной.

In [3]:
data_train, data_test = train_test_split(ratings)

Рассматриваемые подходы обладают структурными параметрами, значения которых необходимо подбирать кросс-валидацией.
Для этого воспользуемся техникой <b>hold-out</b>, выделив из обучающей выборки <b>20%</b> оценок пользователей под валидацию.

In [4]:
X_train, X_valid = train_test_split(data_train)

----

## Эксперименты

### Content-based

Данный подход известен в литературе также под названием когнитивная фильтрация -- метод, основанный на сравнении характеристик
товаров и профиля каждого пользователя.

Как нетрудно заметить, задача сводится к генерации признаков, описывающих профили пользователей и товаров.
Как будет показано ниже, состав признаков имеет прямое влияние на качество модели.

<b>Набор базовых признаков:</b>
    1. Категориальные признаки: 
        * Возраст пользователя
        * Профессия пользователя 
        * Пол пользователя
    2. Наборы булевых признаков: 
        * Жанры фильмов
        * Нормированные средние оценки пользователя в пространстве жанров (1)
    3. Вещественные признаки:
        * Средний рейтинг пользователя
        * Средний рейтинг фильма
        * Константа 1
  
(1):
    
$(u_g * m_g) / n_g$,

$u_g$ -- вектор средних оценок пользователя в пространстве жанров

$m_g$ -- булевый вектор для фильма в пространстве жанров

$n_g$ -- количество жанров, указанных для фильма

Полученные признаки обозначим $\{g_{ui}^n\}_{n=1..N}$

Будем искать рейтинг как линейную комбинацию числовых признаков: 
$\hat{r}_{ui} = \sum_{n=1}^{N} g_{ui}^n \theta_n$.
А значит, категориальные признаки лучше закодировать набором булевых векторов по одному на каждое значение признака.

Для настройки весов признаков воспользуемся <b>Ridge-регрессией</b>: $\underset{w}{min\,} {{|| X w - y||_2}^2 + \alpha {||w||_2}^2}$

Оценим время работы и качество рекомендаций на базовом наборе признаков.

In [28]:
time_start = time.clock()
data_train_cat, data_test_cat = build_categorical(data_train, data_test, movies, users)
target = 'Rating'
predictors = [f for f in data_train_cat.columns if f not in ['UserID', target]]

alpha = 0.2
y_pred = RidgeRegression(alpha).fit(data_train_cat[predictors], data_train_cat[target]).predict(data_test_cat[predictors])

display(pd.DataFrame({'Content_based(BaseFeatures)':
                               {'MSE(Test)': mean_squared_error(data_test_cat[target], y_pred),
                                'Time(sec)': time.clock() - time_start}
                      }).T)

,MSE(Test),Time(sec)
Content_based(BaseFeatures),1.154958,112.302147


Добавим в модель <b>3 дополнительных признака</b>:
    * Средний рейтинг фильмов, выпущенных до и после 1950-х
    * Число токенов в заголовке
    * Доля не стоп-слов в заголовке

In [13]:
time_start = time.clock()
data_train_cat, data_test_cat = build_categorical(data_train, data_test, movies, users, only_base_features=0)
target = 'Rating'
predictors = [f for f in data_train_cat.columns if f not in ['UserID', target]]

alpha = 0.2
y_pred = RidgeRegression(alpha).fit(data_train_cat[predictors], data_train_cat[target]).predict(data_test_cat[predictors])
display(pd.DataFrame({'Content_based(AdditionalFeatures)':
                               {'MSE(Test)': mean_squared_error(data_test_cat[target], y_pred),
                                'Time(sec)': time.clock() - time_start}
                      }).T)

,MSE(Test),Time(sec)
Content_based(AdditionalFeatures),1.137695,116.049525


Качество улучшилось.
Подберем теперь коэффициент регуляризации при помощи кросс-валидации в модели <b>Ridge-регрессии</b>.

In [10]:
X_train_cat, X_valid_cat = build_categorical(X_train, X_valid, movies, users, only_base_features=0)
alpha = [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]
best_score =  cross_val_score(
                   RidgeRegression,
                   alpha,
                   X_train_cat[predictors],
                   X_train_cat[target],
                   X_valid_cat[predictors],
                   X_valid_cat[target])
print (best_score)

{'error': 0.89793568592919681, 'best_param': 1000.0}


Зафиксируем $\alpha=1000.0$. Оценим качество на тестовой выборке.

In [14]:
res = {}
time_start = time.clock()
data_train_cat, data_test_cat = build_categorical(data_train, data_test, movies, users, only_base_features=0)
alpha = 1000.0
y_pred = RidgeRegression(alpha).fit(data_train_cat[predictors], data_train_cat[target]).predict(data_test_cat[predictors])
res['Content_based'] =  {'MSE(Test)': mean_squared_error(data_test_cat[target], y_pred),
                         'Time(sec)': time.clock() - time_start}
display(pd.DataFrame(res).T)

,MSE(Test),Time(sec)
Content_based,0.969475,117.940782


------

## Neighborhood

Альтерантивой <b>content-based</b> подходу может служить <b>Collaborative filtering</b> -- метод, рекомендации, при
котором анализируется реакция пользователя на товары. Имя матрицу <b>user-item</b> из оценок пользователей можно определить меру похожести товаров

$$\textbf{AdjustedCosineSimilarity(i, j)} = \frac{\sum_{u \in U}(r_{ui} - \hat{r_u})(r_{uj} - \hat{r_u})}{\sqrt{\sum_{u \in U}(r_{ui} - \hat{r_u})^2} \sqrt{\sum_{u \in U}(r_{uj} - \hat{r_u})^2}}$$

$U$ -- множество пользователей, которые оценили фильмы $i, j$

$\hat{r_{u}}$ -- средний рейтинг пользователя $u$

<b>Рейтинги неизвестных фильмов</b> считаются по формуле:
$$\hat{r_{ui}} = \frac{\sum_{j : r_{uj} \neq 0} sim(i, j) r_{uj}} {\sum_{j : r_{uj} \neq 0} sim(i, j)}$$

Для обеспечения устойчивости вычислений положим отрицательные значения <b>AdjustedCosineSimilarity(i, j)</b> равными 0.

В данной части исследования нам потребуется фреймворк <b>Apache Hadoop</b>, обеспечивающий высокую производительность, реализуя
потоковую обработку больших объемов данных с использованием метода параллельных вычислений <b>Map-Reduce</b>.

Все исследования проводились на кластере <b>Amazon</b>. Все реализующие данный подход модули собраны в приложении к 
ноутбуку с отчетом.Здесь продемонстрируем итоговые результаты.

<b>distance_matrix.npy</b> -- хранит расстояния между товарами

In [1]:
!python3 neighborhood.py data_train.csv  data_test.csv distance_matrix.npy

MSE(Train):1.3273260813429637, N:1
MSE(Test):1.7566472874917882, N:1
--------------------------------------------------------------------
MSE(Train):2.5190571576844105, N:5
MSE(Test):1.9604546285273967, N:5
--------------------------------------------------------------------
MSE(Train):3.937038550537882, N:10
MSE(Test):2.220438525865518, N:10
--------------------------------------------------------------------
MSE(Train):5.448347744554113, N:20
MSE(Test):2.5875742772325157, N:20
--------------------------------------------------------------------
MSE(Train):7.905597938221867, N:50
MSE(Test):3.3618900375893426, N:50
--------------------------------------------------------------------
MSE(Train):11.93320154733641, N:100
MSE(Test):4.79316476579518, N:100
--------------------------------------------------------------------
MSE(Train):13.739968762456785, N:1000
MSE(Test):14.920316521034719, N:1000
--------------------------------------------------------------------
MSE(Train):13.74955813667

In [6]:
print ('Neighborhood')
df = pd.read_csv('neghborhood.csv', index_col=0)
display(pd.DataFrame(df))

Neighborhood


,N,MSE(Test,MSE(Train)
0,1,1.75664,1.3273
1,5,1.96040,2.5190
2,10,2.22040,3.9370
3,20,2.58750,5.4483
4,50,3.36180,7.9055
5,100,4.79310,11.9332
6,1000,14.92030,13.7399
7,5000,14.92030,13.7399


Видно, что с ростом длины списка похожих товаров качество ухудшается как на обучении, так и на контроле.
А значит достаточно хранить небольшое количество самых похожих товаров для осуществления качественного прогноза.

Время работы (<b>~2000</b> секунд) для разных <b>N</b> примерно одинаково, так как основная вычислительная сложность 
приходится на вычисление матрицы похожести товаров. Далее, пользуемся эффективными матричными вычислениями.

---

Также приведем формулы для еще одного подхода в коллаборативной фильтрации.

Симметричным подходом в коллаборативной фильтрации является <b>user-based</b>.

$$\textbf{AdjustedCosineSimilarity(u, v)} = \frac{\sum_{i \in I}(r_{ui} - \hat{r_i})(r_{vi} - \hat{r_i})}{\sqrt{\sum_{i \in I}(r_{ui} - \hat{r_i})^2} \sqrt{\sum_{i \in I}(r_{vi} - \hat{r_i})^2}}$$

$I$ -- множество фильмов, которые были оценены пользователями $u, v$

$r_{i}$ -- средний рейтинг фильма $i$

-----

## Latent factor

В данном подходе предлагается найти пространство признаков, в котором мы сможем представить фильмы и пользователей, а также оценить меру их близости (рейтинг фильма, поставленный пользователем).

<b>Введем обозначения:</b>
    
$K$ -- число латентных признаков

$U$ -- пространство пользователей

$I$ -- пространство фильмов

$p_u$ и $q_i$ -- представления пользователя $u$ и фильма $i$ соответственно в пространстве латентных признаков $R^K$

$P = (p_u)_{u \in  U}\in R^{|U| * K}$

$Q = (q_i)_{i \in  I} \in R^{|I| * K}$

$Q[u] \in R^{n_u * K}$ -- подматрица матрицы $Q$ только для товаров, оцененных пользователем $u$.

$P[i] \in R^{n_i * K}$ -- подматрица матрицы $P$ только для пользователей, оценивших товар $i$.

В качестве меры близости рассмотрим скалярное произведение:

$\hat{r}_{ui} = p^T_u q_i$.

Для настройки модели будем минимизировать следующий функционал:
$\sum_{(u, r, r_{ui})} (r_{ui} - p^T_{u} q_i)^2 + \lambda_p p^T_{u} p_{u} + \lambda_q q^T_{i} q_{i}$   

<b>Шаг перенастройки $p_u$ при фиксированной матрице $Q$:</b>
    
$A_u = Q[u]^T Q[u]$

$d_u = Q[u]^T r_u$

$p_u = (\lambda_p n_u I + A_u)^{-1} d_u$

<b>Шаг перенастройки $q_i$ при фиксированной матрице $P$:</b>
    
$A_i = P[i]^T P[i]$

$d_u = P[i]^T r_i$

$q_i = (\lambda_q n_i I + A_i)^{-1} d_i$

Зафиксируем значения структурных параметров модели:
    
$\lambda_p = 0.2$

$\lambda_q = 0.001$

Начальная инициализация матриц $Q, P$ -- рандомные значения  на отрезке [0.0, 0.1).
                                                                         
Подберем размерность пространства латентных признаков $K$ при помощи процедуры кросс-валидации.

In [31]:
res = {}
KList = [5, 10, 30, 50, 80, 100] 
N = 5
for K in KList:
    time_start = time.clock()
    P, Q = matrix_factorization(X_train.reset_index(), movies.MovieID.values, users.UserID.values, K, N)
    ExecTime = time.clock() - time_start
    
    answer = np.zeros_like(X_valid.Rating.values)
    for i, (uid, mid) in enumerate(X_valid[['UserID', 'MovieID']].values):
        answer[i] = P[uid].dot(Q[mid])
    error_valid = mean_squared_error(X_valid['Rating'].values, answer)
    
    answer = np.zeros_like(X_train.Rating.values)
    for i, (uid, mid) in enumerate(X_train[['UserID', 'MovieID']].values):
        answer[i] = P[uid].dot(Q[mid])
    error_train = mean_squared_error(X_train['Rating'].values, answer)
    res[K] = {'MSE(Train)': error_train, 'MSE(Test)': error_valid, 'Time(sec)': ExecTime}
    
res = pd.DataFrame(res).T.reset_index()
res.columns = ['K', 'MSE(Test)', 'MSE(Train)', 'Time(sec)']
print ('Latent factor')
display(res)

Latent factor


,K,MSE(Test),MSE(Train),Time(sec)
0,5,0.812049,0.658603,72.117384
1,10,0.805715,0.596706,756.382538
2,30,0.817185,0.491476,1358.085426
3,50,0.821310,0.444733,1428.335245
4,80,0.821851,0.409215,1526.109554
5,100,0.822368,0.396922,1634.068032


Видно, что с ростом числа латентных признаков, модель переобучается.

Очевидным фактом также является увеличение алгоритмической сложности модели и объема используемой памяти.

Зафиксируем оптимальное значение $K = 10$. Увеличим число итераций до сходимости $N = 20$.

Оценим работу на тестовой выборке.

In [32]:
time_start = time.clock()
K = 10
N = 20
P, Q = matrix_factorization(data_train.reset_index(), movies.MovieID.values, users.UserID.values, K, N)

answer = np.zeros_like(data_test.Rating.values)
for i, (uid, mid) in enumerate(data_test[['UserID', 'MovieID']].values):
    answer[i] = P[uid].dot(Q[mid])
    
display(pd.DataFrame({'Latent factor':
                               {'MSE(Test)': mean_squared_error(data_test['Rating'].values, answer),
                                'Time(sec)': time.clock() - time_start}
                      }).T)

,MSE(Test),Time(sec)
Latent factor,0.798112,3769.790197


----

## Выводы

In [17]:
pd.read_csv('results.csv', index_col=0)

,MSE(Test),Time(sec)
Content_based,0.969475,117.940782
Latent factor,0.798112,3769.790197
Neighborhood,1.756640,2400.000000


В данном исследовании были рассмотрены <b>контентные</b> и <b>коллаборативные</b> алгоритмы выработки рекомендаций.

Самым лучшим с точки зрения качества оказался <b>Latent factor</b> подход. Самым быстрым --  <b>Content_based</b>.

<b>Latent factor</b> является оптимизационным алгоритмом, поэтому более гибок с точки зрения оптимизирующего функционала.